In [4]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황.csv


In [ ]:
import pandas as pd
raw_df = pd.read_csv(filename, encoding='EUC-KR')
raw_df

In [47]:
import numpy as np
import re
from tqdm import notebook

In [126]:
raw_df.head(1)

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.55014,http://parks.seoul.go.kr/template/sub/namsan.do


In [136]:
new_df = pd.concat([raw_df.공원명, raw_df.지역, raw_df.면적, raw_df['Y좌표(WGS84)'], raw_df['X좌표(WGS84)']], axis=1)
new_df

,공원명,지역,면적,Y좌표(WGS84),X좌표(WGS84)
0,남산도시자연공원,중구,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,37.550140,126.990377
1,길동생태공원,강동구,80683㎡,37.540394,127.154779
2,서울대공원,과천시,9132690m²,37.426449,127.019847
3,서울숲,성동구,480994㎡,37.543072,127.041798
4,월드컵공원,마포구,2284085㎡,37.571805,126.878907
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635㎡,37.579762,127.095997
128,문화비축기지,마포구,140022㎡,37.571718,126.893246
129,경춘선숲길,NaN,211392㎡,37.627077,127.076482
130,율현공원,강남구,157535㎡,37.472332,127.115594


In [137]:
type(new_df['면적'][0])

str

In [138]:
new_df = new_df.fillna(0)
new_df

,공원명,지역,면적,Y좌표(WGS84),X좌표(WGS84)
0,남산도시자연공원,중구,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,37.550140,126.990377
1,길동생태공원,강동구,80683㎡,37.540394,127.154779
2,서울대공원,과천시,9132690m²,37.426449,127.019847
3,서울숲,성동구,480994㎡,37.543072,127.041798
4,월드컵공원,마포구,2284085㎡,37.571805,126.878907
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635㎡,37.579762,127.095997
128,문화비축기지,마포구,140022㎡,37.571718,126.893246
129,경춘선숲길,0,211392㎡,37.627077,127.076482
130,율현공원,강남구,157535㎡,37.472332,127.115594


In [139]:
# 1번문제
m2 = []
for m in new_df.면적:
  m = str(m)
  m = m.split()
  m2.append(m[0])

new_df['면적2'] = m2
new_df.head(6)

,공원명,지역,면적,Y좌표(WGS84),X좌표(WGS84),면적2
0,남산도시자연공원,중구,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,37.550140,126.990377,2896887㎡
1,길동생태공원,강동구,80683㎡,37.540394,127.154779,80683㎡
2,서울대공원,과천시,9132690m²,37.426449,127.019847,9132690m²
3,서울숲,성동구,480994㎡,37.543072,127.041798,480994㎡
4,월드컵공원,마포구,2284085㎡,37.571805,126.878907,2284085㎡
5,광화문시민열린마당,종로구,8948.1㎡,37.575045,126.978023,8948.1㎡


In [140]:
area = new_df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))
new_df['면적2'] = area
new_df

,공원명,지역,면적,Y좌표(WGS84),X좌표(WGS84),면적2
0,남산도시자연공원,중구,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,37.550140,126.990377,2896887.0
1,길동생태공원,강동구,80683㎡,37.540394,127.154779,80683.0
2,서울대공원,과천시,9132690m²,37.426449,127.019847,9132690.0
3,서울숲,성동구,480994㎡,37.543072,127.041798,480994.0
4,월드컵공원,마포구,2284085㎡,37.571805,126.878907,2284085.0
...,...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635㎡,37.579762,127.095997,109635.0
128,문화비축기지,마포구,140022㎡,37.571718,126.893246,140022.0
129,경춘선숲길,0,211392㎡,37.627077,127.076482,211392.0
130,율현공원,강남구,157535㎡,37.472332,127.115594,157535.0


In [141]:
# 2번문제
size1, size2 = [],[]

for i in new_df.면적2:
  if i < 100000:
    size1.append('소형')
    size2.append(3)
  elif 100000 < i < 1000000:
    size1.append('중형')
    size2.append(7)
  elif 1000000 < i:
    size1.append('대형')
    size2.append(15)
  else:
    size1.append('')
    size2.append('')

In [142]:
new_df['분류'] = size1
new_df['크기'] = size2
new_df

,공원명,지역,면적,Y좌표(WGS84),X좌표(WGS84),면적2,분류,크기
0,남산도시자연공원,중구,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,37.550140,126.990377,2896887.0,대형,15
1,길동생태공원,강동구,80683㎡,37.540394,127.154779,80683.0,소형,3
2,서울대공원,과천시,9132690m²,37.426449,127.019847,9132690.0,대형,15
3,서울숲,성동구,480994㎡,37.543072,127.041798,480994.0,중형,7
4,월드컵공원,마포구,2284085㎡,37.571805,126.878907,2284085.0,대형,15
...,...,...,...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635㎡,37.579762,127.095997,109635.0,중형,7
128,문화비축기지,마포구,140022㎡,37.571718,126.893246,140022.0,중형,7
129,경춘선숲길,0,211392㎡,37.627077,127.076482,211392.0,중형,7
130,율현공원,강남구,157535㎡,37.472332,127.115594,157535.0,중형,7


In [143]:
# 3번문제
df = pd.concat([new_df.공원명, new_df.지역, new_df['면적2'], new_df['Y좌표(WGS84)'], new_df['X좌표(WGS84)'], new_df.분류, new_df.크기], axis=1)
df

,공원명,지역,면적2,Y좌표(WGS84),X좌표(WGS84),분류,크기
0,남산도시자연공원,중구,2896887.0,37.550140,126.990377,대형,15
1,길동생태공원,강동구,80683.0,37.540394,127.154779,소형,3
2,서울대공원,과천시,9132690.0,37.426449,127.019847,대형,15
3,서울숲,성동구,480994.0,37.543072,127.041798,중형,7
4,월드컵공원,마포구,2284085.0,37.571805,126.878907,대형,15
...,...,...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635.0,37.579762,127.095997,중형,7
128,문화비축기지,마포구,140022.0,37.571718,126.893246,중형,7
129,경춘선숲길,0,211392.0,37.627077,127.076482,중형,7
130,율현공원,강남구,157535.0,37.472332,127.115594,중형,7


In [144]:
df.rename(columns = {'공원명' : '공원명','지역':'지역','면적2':'면적','Y좌표(WGS84)':'위도','X좌표(WGS84)':'경도','분류':'분류','크기':'크기'}, inplace = True)
df

,공원명,지역,면적,위도,경도,분류,크기
0,남산도시자연공원,중구,2896887.0,37.550140,126.990377,대형,15
1,길동생태공원,강동구,80683.0,37.540394,127.154779,소형,3
2,서울대공원,과천시,9132690.0,37.426449,127.019847,대형,15
3,서울숲,성동구,480994.0,37.543072,127.041798,중형,7
4,월드컵공원,마포구,2284085.0,37.571805,126.878907,대형,15
...,...,...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,109635.0,37.579762,127.095997,중형,7
128,문화비축기지,마포구,140022.0,37.571718,126.893246,중형,7
129,경춘선숲길,0,211392.0,37.627077,127.076482,중형,7
130,율현공원,강남구,157535.0,37.472332,127.115594,중형,7


In [100]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving kakaoapikey.txt to kakaoapikey.txt


In [145]:
with open(filename) as f:
  api_key = f.read()

In [146]:
import requests
from urllib.parse import quote

In [147]:
keyword = '합정 스타벅스'
serch_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
url = f'{serch_url}?query={quote(keyword)}'
# Authorization: KakaoAK {REST_API_KEY}
result = requests.get(url,
            headers={"Authorization": f'KakaoAK {api_key}'}).json()

In [148]:
result['documents'][0]

{'address_name': '서울 마포구 서교동 395-166',
 'category_group_code': 'CE7',
 'category_group_name': '카페',
 'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
 'distance': '',
 'id': '26572121',
 'phone': '1522-3232',
 'place_name': '스타벅스 서교점',
 'place_url': 'http://place.map.kakao.com/26572121',
 'road_address_name': '서울 마포구 양화로 78',
 'x': '126.916980454434',
 'y': '37.5514601750423'}

In [149]:
import numpy as np
import re
from tqdm import notebook
import folium

In [150]:
location = [37.541, 126.986]
map = folium.Map(location=location, zoom_start=12)
map

In [157]:
location = [37.541, 126.986]
map = folium.Map(location=location, zoom_start=12)

for i in df.index:
  folium.CircleMarker(
      radius=int(df.크기[i]),                                   # int64는 처리 불가
      location=[df.위도[i], df.경도[i]],
      popup=folium.Popup(f'{df.공원명[i]}', max_width=300),   # popup 누르면 나오는 문구, max_widht 팝업창 넓이
      tooltip=f'{df.공원명[i]},({df.면적[i]}㎡)',   # 커서온시 문구
      color='crimson',    # 마커 색
      fill=True,   # 마커 채우기
      fill_color='crimson'
  ).add_to(map)

title = '<h3 align="center" style="font-size:20px">서울시 공원 위치 및 크기</h3>'
map.get_root().html.add_child(folium.Element(title))
map